In [25]:
# first we get the unofficial election results:

import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the NYT webpage containing the data
url = 'https://www.nytimes.com/interactive/2024/11/05/us/elections/results-new-york-constitutional-amendment-1-equal-protection-of-law.html'  # Replace with the actual URL

# send a GET request to the webpage
response = requests.get(url)

# 200 status code = YES, this went well ...
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the correct table (adjust selector as needed)
    tables = soup.find_all('table', {'data-rid': 'NY-I-83165-NY-2024-11-05'})

    if len(tables) >= 2:  # check if at least 2 such tables exist
        table = tables[1]  # select the second table (it's the 2nd one on the page)

    # extract rows
    rows = []
    # tr, td, th are all html tags: <tr> = table row, <td> = table cell ...
    for row in table.find_all('tr'):
        cells = row.find_all(['td', 'th'])  # het all cells
        row_data = [cell.text.strip() for cell in cells]
        if row_data:  # skip empty rows
            rows.append(row_data)

    # use the first row as headers and remaining rows as data
    headers = rows[0]  # first row as column names
    data = rows[1:]  # all other rows as data

    data = data[:-1]
    # chop off the last row, it was saying "See fewer" from the page

    # create DataFrame
    df = pd.DataFrame(data, columns=headers)

    # display the DataFrame
    print(df)

    # save as JSON
    json_file_path = '/content/ballot_results.json'  # Update path if needed
    df.to_json(json_file_path, orient='records', indent=4)

    print(f"Data saved to {json_file_path}")

else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")




       County  Yes   No Total votes Percent of votes in % In
0     Suffolk  54%  46%     692,860                     >95%
1    Brooklyn  79%  21%     665,275                     >95%
2      Nassau  55%  45%     628,488                     >95%
3      Queens  74%  26%     586,360                      94%
4   Manhattan  86%  14%     550,873                      80%
..        ...  ...  ...         ...                      ...
57     Seneca  48%  52%      14,327                     >95%
58      Lewis  31%  69%      11,759                     >95%
59      Yates  40%  60%      10,164                     >95%
60   Schuyler  46%  54%       9,080                     >95%
61   Hamilton  39%  61%       3,148                     >95%

[62 rows x 5 columns]
Data saved to /content/ballot_results.json
